In [1]:
import os
from pathlib import Path
import pandas as pd
import plotly.graph_objects as go
import urllib, json
import plotly
import numpy as np
from plotly.offline import iplot
#import plotly.plotly as py

In [2]:
#Read csv files
path = "C:\\Users\\harry\\Github\\CAP\\Capstone\\AIS_results"

shipStats = pd.read_csv(os.path.join(path,"shipStats"+".csv"),dtype=object)
voyageStats = pd.read_csv(os.path.join(path,"voyageStats"+".csv"),dtype=object)
norms = pd.read_csv(os.path.join(path,"norms"+".csv"),dtype=object)
megaStats=pd.read_csv(os.path.join(path,"megaStats"+".csv"),dtype=object)

shipTypes=["drifting_longlines",
"fixed_gear",
"pole_and_line",
"purse_seines",
"trawlers",
"trollers",
"unknown"
]
megaStats['shipNum']=megaStats.index
print("megaStats.shape=",megaStats.shape)
megaStats.columns


megaStats.shape= (354, 19)


Index(['Unnamed: 0', 'numVoy', 'mmsi', 'numObsInVoys', 'type', 'ave_len_voy',
       'ave_time_gap', 'ave_dist_gap', 'numObs', 'numShips', 'numVoyages',
       'aveNumVoyages', 'numVoyages_q25', 'vi_dist_diff_q75',
       'vi_time_diff_q75', 'voy_len_q75', 'time_gap_q75', 'dist_gap_q75',
       'shipNum'],
      dtype='object')

In [81]:
#dff=pd.DataFrame
#print(dff.shape)
#del dff
dff=megaStats.loc[:,['mmsi','type','ave_len_voy','voy_len_q75','ave_time_gap','time_gap_q75','ave_dist_gap','dist_gap_q75','shipNum']]
#dff.reset_index
dff['flow']=1
dff.columns

#Risk Assessment - Voyage Length 
dff.astype({'ave_len_voy':'float', 'voy_len_q75':'float'}).dtypes
#voyLenRisks = {True:'Long Voyage', False:'Nominal Voyage'} 
dff['voyLenRisk']="Not Set"
col4ave=dff.columns.get_loc('ave_len_voy')
col4q75=dff.columns.get_loc('voy_len_q75')
col4risk=dff.columns.get_loc('voyLenRisk')
len(dff)

for s in range(0,len(dff)):
    if (dff.iat[s,col4ave]>dff.iat[s,col4q75]):
        dff.iat[s,col4risk]="Long Voyage"
    else:
        dff.iat[s,col4risk]="Nominal Voyage"

dff.voyLenRisk.describe()  

#Risk Assessment - Time Gap between transmissions 
dff.astype({'ave_time_gap':'float', 'time_gap_q75':'float'}).dtypes
#voyLenRisks = {True:'Long Voyage', False:'Nominal Voyage'} 
dff['timeGapRisk']="Not Set"
col4ave=dff.columns.get_loc('ave_time_gap')
col4q75=dff.columns.get_loc('time_gap_q75')
col4risk=dff.columns.get_loc('timeGapRisk')
len(dff)

for s in range(0,len(dff)):
    if (dff.iat[s,col4ave]>dff.iat[s,col4q75]):
        dff.iat[s,col4risk]="Long Time Gap"
    else:
        dff.iat[s,col4risk]="Nominal Time Gap"

dff.timeGapRisk.describe()  

#Risk Assessment - Time Gap between transmissions 
dff.astype({'ave_dist_gap':'float', 'dist_gap_q75':'float'}).dtypes
#voyLenRisks = {True:'Long Voyage', False:'Nominal Voyage'} 
dff['distGapRisk']="Not Set"
col4ave=dff.columns.get_loc('ave_dist_gap')
col4q75=dff.columns.get_loc('dist_gap_q75')
col4risk=dff.columns.get_loc('distGapRisk')
len(dff)

for s in range(0,len(dff)):
    if (dff.iat[s,col4ave]>dff.iat[s,col4q75]):
        dff.iat[s,col4risk]="Long Dist Gap"
    else:
        dff.iat[s,col4risk]="Nominal Dist Gap"

dff.distGapRisk.describe()  


Index(['mmsi', 'type', 'ave_len_voy', 'voy_len_q75', 'ave_time_gap',
       'time_gap_q75', 'ave_dist_gap', 'dist_gap_q75', 'shipNum', 'flow'],
      dtype='object')

mmsi             object
type             object
ave_len_voy     float64
voy_len_q75     float64
ave_time_gap     object
time_gap_q75     object
ave_dist_gap     object
dist_gap_q75     object
shipNum           int64
flow              int64
dtype: object

354

count             354
unique              2
top       Long Voyage
freq              177
Name: voyLenRisk, dtype: object

mmsi             object
type             object
ave_len_voy      object
voy_len_q75      object
ave_time_gap    float64
time_gap_q75    float64
ave_dist_gap     object
dist_gap_q75     object
shipNum           int64
flow              int64
voyLenRisk       object
dtype: object

354

count               354
unique                2
top       Long Time Gap
freq                230
Name: timeGapRisk, dtype: object

mmsi             object
type             object
ave_len_voy      object
voy_len_q75      object
ave_time_gap     object
time_gap_q75     object
ave_dist_gap    float64
dist_gap_q75    float64
shipNum           int64
flow              int64
voyLenRisk       object
timeGapRisk      object
dtype: object

354

count                  354
unique                   2
top       Nominal Dist Gap
freq                   184
Name: distGapRisk, dtype: object

In [75]:
def genSankey(df,cat_cols=[],value_cols='',title='Sankey Diagram'):
    # maximum of 6 value cols -> 6 colors
    colorPalette = ['#4B8BBE','#306998','#FFE873','#FFD43B','#646464']
    labelList = []
    colorNumList = []
    for catCol in cat_cols:
        labelListTemp =  list(set(df[catCol].values))
        colorNumList.append(len(labelListTemp))
        labelList = labelList + labelListTemp
        
    # remove duplicates from labelList
    labelList = list(dict.fromkeys(labelList))
    
    # define colors based on number of levels
    colorList = []
    for idx, colorNum in enumerate(colorNumList):
        colorList = colorList + [colorPalette[idx]]*colorNum
        
    # transform df into a source-target pair
    for i in range(len(cat_cols)-1):
        if i==0:
            sourceTargetDf = df[[cat_cols[i],cat_cols[i+1],value_cols]]
            sourceTargetDf.columns = ['source','target','count']
        else:
            tempDf = df[[cat_cols[i],cat_cols[i+1],value_cols]]
            tempDf.columns = ['source','target','count']
            sourceTargetDf = pd.concat([sourceTargetDf,tempDf])
        sourceTargetDf = sourceTargetDf.groupby(['source','target']).agg({'count':'sum'}).reset_index()
        
    # add index for source-target pair
    sourceTargetDf['sourceID'] = sourceTargetDf['source'].apply(lambda x: labelList.index(x))
    sourceTargetDf['targetID'] = sourceTargetDf['target'].apply(lambda x: labelList.index(x))
    
    # creating the sankey diagram
    data = dict(
        type='sankey',
        node = dict(
          pad = 15,
          thickness = 20,
          line = dict(
            color = "black",
            width = 0.5
          ),
          label = labelList,
          color = colorList
        ),
        link = dict(
          source = sourceTargetDf['sourceID'],
          target = sourceTargetDf['targetID'],
          value = sourceTargetDf['count']
        )
      )
    
    layout =  dict(
        title = title,
        font = dict(
          size = 10
        )
    )
       
    fig = dict(data=[data], layout=layout)
    return fig

In [84]:
fig=genSankey(dff,cat_cols=['type','voyLenRisk','timeGapRisk','distGapRisk'],value_cols='flow',title='Sankey Diagram Fig2')
iplot(fig)
plotly.offline.plot(fig, validate=False)

'temp-plot.html'

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = shipTypes,
      color = "blue"
    ),
    link = dict(
      source = [0, 1, 0, 2, 3, 3], # indices correspond to labels, eg A1, A2, A2, B1, ...
      target = [2, 3, 3, 4, 4, 5],
      value = [8, 4, 2, 8, 4, 2]
  ))])

fig.update_layout(title_text="Basic Sankey Diagram", font_size=10)
fig.show()

In [ ]:
import plotly.graph_objects as go
import urllib, json
import pandas as pd

#url = 'https://raw.githubusercontent.com/plotly/plotly.js/master/test/image/mocks/sankey_energy.json'
#response = urllib.request.urlopen(url)
#data = json.loads(response.read())
flows = pd.read_json('csvjson.json')
fig = go.Figure(data=[go.Sankey(
    valueformat = ".0f",
    valuesuffix = "TWh",
    node = dict(
      pad = 15,
      thickness = 15,
      line = dict(color = "black", width = 0.5),
      label =  data['data'][0]['node']['label'],
      color =  data['data'][0]['node']['color']
    ),
    link = dict(
      source =  data['data'][0]['link']['source'],
      target =  data['data'][0]['link']['target'],
      value =  data['data'][0]['link']['value'],
      label =  data['data'][0]['link']['label']
  ))])

fig.update_layout(
    hovermode = 'x',
    title="Energy forecast for 2050<br>Source: Department of Energy & Climate Change, Tom Counsell via <a href='https://bost.ocks.org/mike/sankey/'>Mike Bostock</a>",
    font=dict(size = 10, color = 'white'),
    plot_bgcolor='black',
    paper_bgcolor='black'
)

fig.show()